In [6]:
import os 
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
data_dir = os.path.join(os.getcwd(), 'data')

## 1. Кластеризация путей процесса

Ранее в лекции мы рассмотрели, что анализировать каждый путь процесса - это дорого и нецелесообразно, лучше пути процесса группировать. <br>
Для определения основных путей процесса (наиболее частые случаи реализации) мы используем подход, основанный на кластеризации временных рядов. <br>
Пусть каждый случай реализации процесса - это временной ряд, каждый этап - это точка временного ряда <br>
Загрузим данные:

In [ ]:
df_ts = pd.read_csv(os.path.join(data_dir, 'dtw_df.csv'))

Выведем путь процесса для одного Id<br>
Процесс последовательно проходит через статусы: 10-20-25-30-33-35-40

In [ ]:
df_ts[df_ts.Id=='0894EF37C23A1ED88DC55B5D17A8294F']

<br>
Перед кластеризацией временных рядов нужно посчитать расстояние между рядами. <br>
Например, это можно сделать через L1/L2-distance, но лучше использовать DTW-алгоритм <br>
Имплементируем его по частям:

In [ ]:
# Преобразуйте исходный набор данных:
# - Трансформируйте датасет, чтобы каждая строка содержала (Id, Status_1...Status_n) - pandas.crosstab()
# - Значения - это сумма Timediff 
# Не забудьте сбросить индекс в новом dataframe

transformed_df = # Ваш код #
cols = transformed_df.columns.values[1:]

In [ ]:
# Для примера выберем два ряда:
x1 = transformed_df[transformed_df.Id=='645106F183B01EE88EAE80635E4F20D3'].fillna(-10).values[0,1:].astype(np.float16)
x2 = transformed_df[transformed_df.Id=='0894EF37C23A1ED88F9307ED42AE579B'].fillna(-10).values[0,1:].astype(np.float16)

In [ ]:
# Построим их графики 
fig, ax = plt.subplots()
plt.plot(x1.T, 'b', label='x1')
plt.plot(x2.T, 'g', label='x2')
ax.set_xticklabels(cols)
ax.set_xticks([x for x in range(len(cols))])
plt.legend()

<br>
Для расчета расстояний методом DTW нужно выполнить три шага:
- Посчитать расстояние между рядами
- Вычислить accumulated loss
- Вычислить backtrack по матрице accumulated loss

#### Посчитаем L2-расстояние

In [ ]:
def L2_dist(x1, x2):
    distances = np.zeros((len(x1), len(x2)))
  
    # Реализуйте алгоритм, высчитывающий попарное L2-расстояние для всех точек двух временных рядов
    #### Ваш код ####

    return distances


In [ ]:
d = L2_dist(x1, x2)

In [ ]:
fig_dims = (5, 4)
fig, ax = plt.subplots(figsize=fig_dims)
ax.set(title='Матрица расстояний между временными рядами')
sns.heatmap(d, vmin=0, vmax=np.max(d))
plt.show()

#### Вычислим accumulaed loss

Напомню, что accumulated loss считается по формуле D<sub>i,j</sub>=d<sub>i,j</sub>+min{D<sub>i,j</sub>, D<sub>i−1,j</sub>, D<sub>i,j−1</sub>}

In [ ]:
def D_loss(dist_matrix):
    D_loss = np.zeros((dist_matrix.shape[0], dist_matrix.shape[1]))
    
    #### Ваш код ####
    
    return D_loss

In [ ]:
D = D_loss(d)

In [ ]:
fig_dims = (5, 4)
fig, ax = plt.subplots(figsize=fig_dims)
ax.set(title='Накопленный loss между рядами')
sns.heatmap(D, vmin=0, vmax=np.max(D))
plt.show()

#### Вычислим DTW-path

DTW-path - это путь из D<sub>M,N</sub> в D<sub>1,1</sub>. На каждом шаге переход осуществляется по элементам с минимальным accumulated loss: min{D<sub>i,j</sub>, D<sub>i−1,j</sub>, D<sub>i,j−1</sub>}

In [ ]:
def DTW(accum_loss):
    #### path - оптимальный путь
    #### transform_cost - затраты на преобразование ряда
    #### Ваш код ####
    return path, transform_cost

In [ ]:
path, transform_cost = DTW(x1, x2, D, d)

In [ ]:
fig_dims = (5, 4)
fig, ax = plt.subplots(figsize=fig_dims)
ax.set(title='DTW + Накопленный loss')
sns.heatmap(D, vmin=0, vmax=np.max(D))
plt.plot([x[0] for x in path], [x[1] for x in path])

## 2. Выбор наиболее влияющих факторов

Ещё один пример применения машинного обучения в анализе процессов - это определение факторов, которые влияют на время выполнения процесса.<br>
Рассмотрим несколько примеров для разного класса моделей

Испортируем полезные библиотеки и загрузим исходный датасет

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import RFE, SelectKBest, chi2
import seaborn as sns

features_df = pd.read_csv(os.path.join(data_dir, 'dataset.csv'), error_bad_lines=False)

LABEL - это класс поломки (всего 5 классов), его надо предсказать<br>
все остальные поля - признаки<br>

Разобьем датасет на features и target:

In [9]:
X = features_df.drop('LABEL', axis=1)
y = features_df.LABEL

Большинство полей - категориальные признаки, их нужно закодировать через label encoder или one-hot encoder<br>

In [146]:
from sklearn.preprocessing import LabelEncoder

X_labels = ### label encoded представление ### 
X_ohe =    ### one-hot encoded encoded представление ### 

Для оценки качества модели будем использовать 20% от выборки  <br>

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X_labels, y, test_size=0.2, random_state=0)
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(X_ohe, y, test_size=0.2, random_state=0)

<br>
Обучим линейную модель <br>

In [ ]:
clf_lg = ### Логистическая регрессия. Обратите внимание, что вы решаете задачу с несколькиими классами ###
clf_lg.fit(X_train_ohe, y_train_ohe)
y_pred_clf = clf_lg.predict(X_test_ohe)
print(metrics.classification_report(y_test_ohe, y_pred_clf))

Вклад каждого фактора может быть интерпретировано как соответствующий коэффициент при признаке <br>

In [ ]:
lg_scores = np.append(clf_lg.intercept_, clf_lg.coef_)
lg_df = pd.DataFrame(list(zip(X_train_ohe.columns, abs(lg_scores))), columns=['Фактор', 'Score_rf'])
print(lg_df.sort_values('Score_rf', ascending=False).head(10))

В случае, если мы используем Random Forest, то получить оценку вклада факторов можно через меру <i>mean impurity decrease</i> <br>

In [ ]:
clf_rf =  ### Fit Random Forest to train data ###

rf_scores = clf_rf.feature_importances_
rf = pd.DataFrame(list(zip(X.columns, rf_scores)),columns=['Фактор', 'Score_rf'])

### Если необходимо, нормализуйте величины Score_rf, записанные в dataframe ###

y_pred = clf.predict(X_test)

print(metrics.classification_report(y_test,y_pred))

### Отсортируйте датафрейм rf по Score_rf и выведите top-5 наиболее важных признаков ###

В случае с линейными моделями или tree-методами получить важность признаков довольно просто.<br>
Но если вы используете SVM с нелинейным ядром, то результаты будут неинтерпретируемы.<br>
В этом случае можно использовать Permutation Importance, который реализован в модуле eli5

In [48]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)

In [ ]:
perm = PermutationImportance(### Ваш код ###)
perm.fit(### Ваш код ###)
eli5.show_weights(### Ваш код ###)
    
### Выведите top-5 наиболее важных признаков ###